In [1]:
import csv
import json
import pandas as pd
import re
import logging
import os
import random
import sys
from heapq import nlargest
import openai
import torch
from tenacity import retry, stop_after_attempt, wait_exponential
import pandas as pd


with open('data_set_v1.0/author_json.json', 'r',encoding='utf-8') as f:
    authors = json.load(f)
print(len(authors))

with open('data_set_v1.0/paper_json.json', 'r',encoding='utf-8') as f:
    papers = json.load(f)
print(len(papers))
# sampled papers to 500 
count = 0
max_paper = 100
author_paper_count = []
paper_author_count = []
papers_sampled = {}
authors_sampled = {}
for i in papers:
    if count < 100:
        flag = True
        for author in papers[i]["authors"]:
            if len(authors[author]["papers"])<5:
                flag = False
        if flag and len(papers[i]["authors"]) > 1:
            paper_author_count.append(len(papers[i]["authors"]))
            papers_sampled[i] = papers[i]
            papers_sampled[i]["author_history"] = {}
            
            for author in papers[i]["authors"]:
                papers_sampled[i]["author_history"][author] = {}
                papers_sampled[i]["author_history"][author]["authors_history_text"] = [papers[i]["abstract"]]
                papers_sampled[i]["author_history"][author]["authors_history_dict"] = [{"Abstract":"","Title":""}]
                paper_count_tmp = 0
                for paper_tmp in authors[author]["papers"]:
                    if paper_count_tmp < max_paper and paper_tmp["title"] != papers[i]["title"]:
                        papers_sampled[i]["author_history"][author]["authors_history_text"].append("Abstract: "+paper_tmp["abstract"]+"\nTitle: "+paper_tmp["title"])
                        papers_sampled[i]["author_history"][author]["authors_history_dict"].append({"Abstract":paper_tmp["abstract"],"Title":paper_tmp["title"]})
                        paper_count_tmp = paper_count_tmp + 1
                author_paper_count.append(len(authors[author]["papers"]))
                authors_sampled[author] = authors[author]
            count = count + 1
print("papers count after sampled: ",len(papers_sampled))
print("authors count after sampled: ",len(authors_sampled))

7931
3275
papers count after sampled:  100
authors count after sampled:  243


In [4]:
import csv
import json
import pandas as pd
import re
import logging
import os
import random
import sys
from heapq import nlargest
import openai
import torch
from tenacity import retry, stop_after_attempt, wait_exponential
import pandas as pd

error_list=[]
i = 0
openai.api_key = "31b8638c9eea48709a596501490f9e88"
openai.api_base = "https://gersteinbiocodeeval-eastus2.openai.azure.com/"
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future
key_bundles = [
    ('31b8638c9eea48709a596501490f9e88', "https://gersteinbiocodeeval-eastus2.openai.azure.com/"),
    ('31b8638c9eea48709a596501490f9e88', "https://gersteinbiocodeeval-eastus2.openai.azure.com/"),
    ('31b8638c9eea48709a596501490f9e88', "https://gersteinbiocodeeval-eastus2.openai.azure.com/")
]
    # prompt_construction v1.6
def construct_prompt_cot_agg(paper):

    prompt_str = ""
    prompt_str = prompt_str + "Assuming you’re the leading author of the following research paper, which contains title, abstract and a snippet of introduction. Please extract the research questions from it:\n"
    prompt_str = prompt_str + "Title:" + paper["title"] + "\n"
    prompt_str = prompt_str + "Abstract:" + paper["abstract"] + "\n"

    prompt_str = prompt_str + "Please extract top 3 research questions from the above information and make it a list"+"\n"
    prompt_str = prompt_str + "Research question: "

    return prompt_str


def run_openai(prompt,key_bundles,i,indice):
    my_dict = {}
    my_dict["role"] = "user"
    my_dict["content"] = prompt
    l = []
    l.append(my_dict)
    result = ""
    try:
        key_bundle = key_bundles[i%3]
        openai.api_key, openai.api_base = key_bundle
        response = openai.ChatCompletion.create(engine="gpt-4", messages=l, temperature=0.0,request_timeout=30)
        result = response.choices[0].message["content"].replace('\n', ' ')
        # print(indice+"\t"+result+"\n")
        return result
    except openai.error.Timeout:
        i = i+1
        print("Timeout",indice)
        key_bundle = key_bundles[i%3]
        openai.api_key, openai.api_base = key_bundle
        try:
            response = openai.ChatCompletion.create(engine="gpt-4", messages=l, temperature=0.0,request_timeout=30)
            result = response.choices[0].message["content"].replace('\n', ' ')
            # print(indice+"\t"+result+"\n")
            return result
        except openai.error.Timeout:
            i = i+1
            print("Timeout",indice)
            key_bundle = key_bundles[i%3]
            openai.api_key, openai.api_base = key_bundle
            try:
                response = openai.ChatCompletion.create(engine="gpt-4", messages=l, temperature=0.0,request_timeout=30)
                result = response.choices[0].message["content"].replace('\n', ' ')
                # print(indice+"\t"+result+"\n")
                return result
            except Exception as e:
                print(f"An error occurred: {str(e)}")
                error_list.append(i)
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            error_list.append(i)
        #print(result)
    except openai.error.InvalidRequestError:
        print("InvalidRequestError",indice)
        error_list.append(i)
        #print(result)
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        error_list.append(i)
    return result

id2rq = {}
with open("naive_method_v1.0/paper_research_questions.tsv", "wb", buffering=0) as out_file:
    for indice,paperId in enumerate(papers_sampled):
        prompt = construct_prompt_cot_agg(papers_sampled[paperId])
        # print(prompt)
        # break
        result = run_openai(prompt,key_bundles,i,indice)
        print(indice, result)
        id2rq[paperId] = result
        write_str = bytes(str(paperId)+"\t"+result+"\n", 'utf-8')
        out_file.write(write_str)
        

0 1. How can model-based approaches improve the daily practice of software professionals through Model-Driven Software Engineering (MDSE) or Model-Driven Engineering (MDE)? 2. What are the basic principles and techniques of MDSE that can be quickly understood and applied for immediate benefit? 3. How can the integration of MDSE in existing development processes be effectively achieved, and what are the technical aspects of MDSE, including the building of a domain-specific modeling language and the description of Model-to-Text and Model-to-Model transformations?
1 1. How can a systematic mapping study be effectively conducted in the field of software engineering? 2. What are the key differences between systematic maps and systematic reviews in software engineering, and how can these differences guide the choice between them? 3. What are the guidelines for conducting systematic maps in software engineering based on the comparison with systematic reviews?
2 1. What are the main issues and

In [9]:
id2rq = pd.read_csv('data_set_v2.1/paper_research_questions.tsv', sep='\t', header=None, names=["paperId","rq"])

for i in range(len(id2rq["rq"])):
    content = id2rq["rq"][i]
    paperId = id2rq["paperId"][i]
    temp_res = []
    temp_res.append(content.split("1. ")[1].split("2. ")[0].strip())
    temp_res.append(content.split("2. ")[1].split("3. ")[0].strip())
    temp_res.append(content.split("3. ")[1].strip())
    # print(temp_res)
    paperJson[paperId]["research_questions"] = temp_res
    # break

In [1]:
import json
with open('data_set_v2.1/paper_json_sampled_100.json', 'r',encoding='utf-8') as f:
    paperJson = json.load(f)
print(len(paperJson))
with open('data_set_v2.1/author_json_sampled_100.json', 'r',encoding='utf-8') as f:
    authorJson = json.load(f)
print(len(authorJson))

100
536


In [20]:
count = 0
from langdetect import detect
def interstsValidation(interests, DictOfFields):
    updated_interests = []
    for i in interests:
        updated_interests.extend(i.split("/"))
    # language check 
    lan = detect(" ".join(interests))
    # item count check 
    item_count = len(interests)
    return interests

for index,i in enumerate(authorJson):
    DictOfFields = {}
    for paper in authorJson[i]["papers"]:
        if "fieldsOfStudy" in paper and paper["fieldsOfStudy"] != None:
            for field in paper["fieldsOfStudy"]:
                if field not in DictOfFields:
                    DictOfFields[field] = 0
                DictOfFields[field] = DictOfFields[field] +1 
    interests = []
    if "interests" in authorJson[i]:
        interests = authorJson[i]["interests"]
    interests = interstsValidation(interests, DictOfFields)
    count = count + len(authorJson[i]["papers"])

count / len(authorJson)

LangDetectException: No features in text.

'en'